# <div style="text-align: left; font-size: 30px;">Paper Summarization Bot 🤖</div>
<div style="text-align: left; font-size: 20px; color: gray;">Supported by <strong>LangChain</strong> and <strong>IBM Watsonx</strong></div>


### Table of Contents
1. [Load PDF (paper as you want)](#load-pdf)
2. [Connect to IBM Watsonx.ai and Initialize LLM Model](#connect-ibm-watsonx)
3. [Make Semantic Chunking Using Watsonx Embedding Model](#semantic-chunking)
4. [Apply Semantic Chunk](#apply-semantic-chunk)
5. [Map Reduce Using LangChain](#map-reduce)

<a id="load-pdf"></a>
## Load PDF (paper as you want)


In [2]:
# Import libraries
import os
from dotenv import load_dotenv
from langchain.document_loaders import WebBaseLoader
from langchain.chains.summarize import load_summarize_chain
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
from pydantic import field_validator
from langchain import PromptTemplate # Langchain Prompt Template
from langchain.chains import LLMChain, SimpleSequentialChain # Langchain Chains
from langchain.document_loaders import PyPDFLoader
from termcolor import colored

USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\AICA-mentoring-program\venv\Lib\site-packages\ibm_watson_machine_learning\foundation_models\extensions\langchain\llm.py:60: WatsonxLLMDeprecationWarning: ibm_watson_machine_learning.foundation_models.extensions.langchain.WatsonxLLM is deprecated and will not be supported in the future. Please import from langchain-ibm instead.
To install langchain-ibm run `pip install -U langchain-ibm`.
  _raise_watsonxllm_deprecation_warning()


#### PyPDFLoader를 이용하여 문서를 Load

In [3]:
def load_pdf_to_doc(file_name):
    loader = PyPDFLoader(file_name)
    documents = loader.load()
    return documents

pdf_file = "yolo.pdf"
documents = load_pdf_to_doc(f"./{pdf_file}")

print(len(documents))

10


<a id="connect-ibm-watsonx"></a>
## Connect to IBM Watsonx.ai and initialize LLM

In [4]:
import os
# Load the environment variables
load_dotenv()


# region에 따라 주소가 다를 수 있습니다. 주소를 확인해 주세요.
ibm_cloud_url = "https://us-south.ml.cloud.ibm.com" 
project_id=os.environ['PROJECT_ID']
api_key = os.environ['API_KEY']


if api_key is None or ibm_cloud_url is None or project_id is None:
    raise Exception("One or more environment variables are missing!")
else:
    creds = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }

In [5]:
# watsonx model 초기화
params = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.REPETITION_PENALTY: 1.1,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 1024
}

llm_model = Model(
    model_id="meta-llama/llama-3-70b-instruct",
    params=params,
    credentials=creds,
    project_id=project_id
)

print("Done initializing LLM.")

Done initializing LLM.


c:\AICA-mentoring-program\venv\Lib\site-packages\ibm_watson_machine_learning\foundation_models\utils\utils.py:273: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(default_warning_template.format(


In [6]:
# #Check if the llm works
# llm_model.generate("User: hello. nice to meet you\n Agent:")

<a id="semantic-chunking"></a>
## Make Semantic Chunking Using Watsonx Embedding Model
- Semantic chunking options
    - `percentile` (default) — In this method, all differences between sentences are calculated, and then any difference greater than the X percentile is split.

    - `standard_deviation` — In this method, any difference greater than X standard deviations is split.

    - `interquartile` — In this method, the interquartile distance is used to split chunks.


In [7]:
# 임베딩 모델 초기화
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import Embeddings
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames as EmbedParams
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes

embed_params = {
     EmbedParams.TRUNCATE_INPUT_TOKENS: 3,
     EmbedParams.RETURN_OPTIONS: {
     'input_text': True
     }
 }

embedding = Embeddings(
    model_id="intfloat/multilingual-e5-large",
    params=embed_params,
    credentials=Credentials(
        api_key = api_key,
        url = "https://us-south.ml.cloud.ibm.com"),
    project_id=project_id
    )

<a id="apply-semantic-chunk"></a>
## Apply Semantic Chunk


In [8]:
#Semantic Chunk 만들기 
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(embedding, breakpoint_threshold_type="percentile")
#
semantic_chunks = semantic_chunker.create_documents([d.page_content for d in documents])

# # Enable the following if you want to see the content of the semantic chunks
#for semantic_chunk in semantic_chunks:
#    print(semantic_chunk.page_content)
#    print(len(semantic_chunk.page_content))
    
print(len(semantic_chunks))

46


<a id="map-reduce"></a>
## Map Reduce Using LangChain


In [9]:
# 커스텀 LLM 만들기
from langchain.llms.base import LLM
from pydantic import Field

class CustomWatsonLLM(LLM):
    model: object = Field(...)  # Define model as a field

    def __init__(self, model):
        super().__init__(model=model)

    def _call(self, prompt, stop=None):
        # Use the generate method from your model
        response = self.model.generate(prompt)
        return response['results'][0]['generated_text']
    
    @property
    def _llm_type(self):
        return "custom_watson_llm"

# Instantiate your custom LLM
custom_llm = CustomWatsonLLM(llm_model)

# # Now you can use this with LLMChain
# reduce_chain = LLMChain(llm=custom_llm, prompt=reduce_prompt)
# reduce_chain


In [10]:
from transformers import AutoTokenizer
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain
from langchain.chains import LLMChain, StuffDocumentsChain
from time import perf_counter
from langchain.schema import Document
from termcolor import colored


# Map
map_template = """The following is a set of parts within one long paper
{docs}
Based on the list of parts of the paper, please identify the main point in less than 3 sentences.
Start the summarization with "The main point of this section is"
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
print("Init map chain...")
map_chain = LLMChain(llm=custom_llm, prompt=map_prompt)

# Reduce
reduce_template = """The following is set of summaries:
{doc_summaries}
Take these and distill it into a final, consolidated summary of the main themes. 
Start the finalized summarization with "The final summarization of this paper is"
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
print("Init reduce chain...")
reduce_chain = LLMChain(llm=custom_llm, prompt=reduce_prompt)

# Stuff documents using reduce chain
print("Stuff documents using reduce chain...")
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries"
)

# Reduce documents chain
reduce_documents_chain = ReduceDocumentsChain(
    combine_documents_chain=combine_documents_chain,
    collapse_documents_chain=combine_documents_chain,
    token_max=4000
)

# MapReduce chain setup
map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=map_chain,
    reduce_documents_chain=reduce_documents_chain,
    document_variable_name="docs",
    return_intermediate_steps=True,
    verbose=False
)

# Run the map phase separately
print("Running map phase...")
map_results = []
i=1
for chunk in semantic_chunks:
    # Assuming each chunk is a Document object with the paper content
    result = map_chain({"docs": chunk.page_content})
    map_results.append(result)
    print(colored(f"\nIntermediate map result {i}:", "green"))
    print(result['text'])
    print("\n")
    i+=1
print("Running reduce phase...")


try:
    t1_start = perf_counter()
    results = map_reduce_chain(semantic_chunks)
    steps = results["intermediate_steps"]
    output = results["output_text"]
    t1_stop = perf_counter()
    print("Elapsed time for reduce phase:", round((t1_stop - t1_start), 2), "seconds.\n") 

    # Create a box around the final output
    box_width = 100
    print(colored("+" + "-" * box_width + "+", "cyan"))
    print(colored(f"{output}", "cyan"))
    print(colored("+" + "-" * box_width + "+", "cyan"))
    print("\nDone.")
except Exception as e:
    print(e)


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Init map chain...


C:\Users\USER\AppData\Local\Temp\ipykernel_18472\1145923435.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  map_chain = LLMChain(llm=custom_llm, prompt=map_prompt)
C:\Users\USER\AppData\Local\Temp\ipykernel_18472\1145923435.py:32: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  combine_documents_chain = StuffDocumentsChain(
C:\Users\USER\AppData\Local\Temp\ipykernel_18472\1145923435.py:37: LangChainDeprecationWarning: This class is deprecated. Please see the migration guide here for a recommended replacement: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain/
  reduce_documents_chain = ReduceDocumentsChain(
C:\Users\USER\AppData\Local\Temp\ipykernel_18472\1

Init reduce chain...
Stuff documents using reduce chain...
Running map phase...

Intermediate map result 1:
 
The main point of this section is that the authors introduce YOLO (You Only Look Once), a novel approach to object detection that differs from prior work which uses classifiers for detection.



Intermediate map result 2:
 
The main point of this section is that the authors propose a new object detection algorithm called YOLO (You Only Look Once), which frames object detection as a regression problem and predicts bounding boxes and class probabilities directly from full images in one evaluation. This allows for fast and accurate object detection, making it suitable for applications such as self-driving cars and assistive devices.



Intermediate map result 3:
 
The main point of this section is that traditional object detection methods involve multiple complex steps, whereas the authors propose a simpler approach called YOLO (You Only Look Once), which reframes object detection

Token indices sequence length is longer than the specified maximum sequence length for this model (2528 > 1024). Running this sequence through the model will result in indexing errors


Elapsed time for reduce phase: 116.12 seconds.

+----------------------------------------------------------------------------------------------------+
 

The final summarization of this paper is that it introduces YOLO (You Only Look Once), a novel approach to object detection that frames object detection as a regression problem and predicts bounding boxes and class probabilities directly from full images in one evaluation. The authors propose a unified model that integrates all stages of object detection into a single, jointly optimized model, allowing for fast and accurate object detection. The paper highlights the advantages of YOLO over traditional object detection methods, including its ability to make accurate predictions, learn generalizable representations of objects, and reason globally about the full image. The authors also compare YOLO with other state-of-the-art object detection systems, demonstrating its competitive performance on various benchmarks. Overall, the paper pre